In [1]:
import os

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.client_tool import client_tool

import rich
from rich.pretty import pprint

from dotenv import load_dotenv
load_dotenv()

HOST=os.environ["HOST"]
PORT=os.environ["LLAMA_STACK_PORT"]
MODEL_NAME=os.environ["INFERENCE_MODEL"]



In [2]:
import requests

@client_tool
def weathertool(latitude: float, longitude: float) -> float:
    """
    Get current temperature for provided coordinates in celsius

    :param latitude: latitude of the location
    :param longitude: longitude of the location
    :returns: value containing the temparature of the provided coordinates in celsius
    """
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    
    return data['current']['temperature_2m']

In [3]:
def agent_example():
    client = LlamaStackClient(base_url=f"http://{HOST}:{PORT}")
    agent = Agent(
        client, 
        model=MODEL_NAME,
        instructions= "Give an exact answer to the question asked. Use tools for providing answers when needed. Do not use tools if not needed. Think carefully before answering. If the question is complex, break it down into simple steps and execute",
        sampling_params={
        "strategy": {"type": "top_p", "temperature": 0.7, "top_p": 0.95},
        "max_tokens": 2048,
        },
        tools=[weathertool],
    )

    return agent
    


In [4]:
agent=agent_example()
session_id = agent.create_session("test-tool_call_session")
print(f"Created session_id={session_id} for Agent({agent.agent_id})")


# "I am planning a trip to Switzerland, what are the top 3 places to visit?",
#  "what is the temparature in Paris?",
# "When was the last time that the Boston Red Sox won the world series?"


response = agent.create_turn(
    messages=[
        {
            "role": "user",
            "content": "what is the temparature in Paris?",
        }
    ],
    session_id=session_id,
    stream=False
)
rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")
pprint(response)
#print(response.__dict__)  # Shows the internal dictionary

Created session_id=1b78a3d5-2d5b-4208-b75b-1b9a52f75306 for Agent(892f0c0c-5f3c-498e-aa8c-fda75e9440e9)


Agent Answer: The temperature in Paris is approximately 16.6 degrees Celsius.

Turn(
│   input_messages=[
│   │   UserMessage(content='what is the temparature in Paris?', role='user', context=None),
│   │   ToolResponseMessage(
│   │   │   call_id='79d83c24-beeb-4f19-9d7e-09206274b46f',
│   │   │   content='16.6',
│   │   │   role='tool',
│   │   │   tool_name='weathertool'
│   │   )
│   ],
│   output_message=CompletionMessage(
│   │   content='The temperature in Paris is approximately 16.6 degrees Celsius.',
│   │   role='assistant',
│   │   stop_reason='end_of_turn',
│   │   tool_calls=[]
│   ),
│   session_id='1b78a3d5-2d5b-4208-b75b-1b9a52f75306',
│   started_at=datetime.datetime(2025, 4, 15, 10, 46, 37, 747644, tzinfo=TzInfo(-04:00)),
│   steps=[
│   │   InferenceStep(
│   │   │   api_model_response=CompletionMessage(
│   │   │   │   content='',
│   │   │   │   role='assistant',
│   │   │   │   stop_reason='end_of_turn',
│   │   │   │   tool_calls=[
│   │   │   │   │   ToolCall(
│   │   │   │   │   │   arguments={'latitude': 48.8567, 'longitude': 2.3508},
│   │   │   │   │   │   call_id='79d83c24-beeb-4f19-9d7e-09206274b46f',
│   │   │   │   │   │   tool_name='weathertool',
│   │   │   │   │   │   arguments_json=None
│   │   │   │   │   )
│   │   │   │   ]
│   │   │   ),
│   │   │   step_id='e97ae461-9415-49aa-a0e9-263bbd54a0ac',
│   │   │   step_type='inference',
│   │   │   turn_id='cca971f6-b1b1-4016-b5a8-a62d3ac78f5b',
│   │   │   completed_at=datetime.datetime(2025, 4, 15, 10, 46, 50, 5215, tzinfo=TzInfo(-04:00)),
│   │   │   started_at=datetime.datetime(2025, 4, 15, 10, 46, 37, 748554, tzinfo=TzInfo(-04:00))
│   │   ),
│   │   ToolExecutionStep(
│   │   │   step_id='26817f1b-d80f-487c-9c41-74047a663eb5',
│   │   │   step_type='tool_execution',
│   │   │   tool_calls=[
│   │   │   │   ToolCall(
│   │   │   │   │   arguments={'latitude': 48.8567, 'longitude': 2.3508},
│   │   │   │   │   call_id='79d83c24-beeb-4f19-9d7e-09206274b46f',
│   │   │   │   │   tool_name='weathertool',
│   │   │   │   │   arguments_json=None
│   │   │   │   )
│   │   │   ],
│   │   │   tool_responses=[
│   │   │   │   ToolResponse(
│   │   │   │   │   call_id='79d83c24-beeb-4f19-9d7e-09206274b46f',
│   │   │   │   │   content='16.6',
│   │   │   │   │   tool_name='weathertool',
│   │   │   │   │   metadata={}
│   │   │   │   )
│   │   │   ],
│   │   │   turn_id='cca971f6-b1b1-4016-b5a8-a62d3ac78f5b',
│   │   │   completed_at=datetime.datetime(2025, 4, 15, 10, 46, 50, 587851, tzinfo=TzInfo(-04:00)),
│   │   │   started_at=datetime.datetime(2025, 4, 15, 10, 46, 50, 44195, tzinfo=TzInfo(-04:00))
│   │   ),
│   │   InferenceStep(
│   │   │   api_model_response=CompletionMessage(
│   │   │   │   content='The temperature in Paris is approximately 16.6 degrees Celsius.',
│   │   │   │   role='assistant',
│   │   │   │   stop_reason='end_of_turn',
│   │   │   │   tool_calls=[]
│   │   │   ),
│   │   │   step_id='80287a48-68d8-4064-b44a-9f8fab50516d',
│   │   │   step_type='inference',
│   │   │   turn_id='cca971f6-b1b1-4016-b5a8-a62d3ac78f5b',
│   │   │   completed_at=datetime.datetime(2025, 4, 15, 10, 46, 52, 290250, tzinfo=TzInfo(-04:00)),
│   │   │   started_at=datetime.datetime(2025, 4, 15, 10, 46, 50, 600789, tzinfo=TzInfo(-04:00))
│   │   )
│   ],
│   turn_id='cca971f6-b1b1-4016-b5a8-a62d3ac78f5b',
│   completed_at=datetime.datetime(2025, 4, 15, 10, 46, 52, 302549, tzinfo=TzInfo(-04:00)),
│   output_attachments=[]
)